In [15]:
# import the libraries
import requests
import csv
from bs4 import BeautifulSoup
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
from sklearn import manifold
from sklearn.metrics import euclidean_distances
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt 

### Get brand names

In [26]:
# convert model names to brands

# open the models file and use that to replace model names with brand names. 
# format of file is brand, model
#file = open('models.csv', 'r')
file = open('models.csv', 'r', encoding='mac_roman', newline='')

csv_file = csv.reader(file)

# store the brand names
brands = []
for i in csv_file:
    brands.append(i)
#     parsedData = parsedData.replace(i[1], i[0])

print('Finished storing this csv file into a list')
# close the file
file.close()

Finished storing this csv file into a list


In [27]:
unique_brands = []
for b in brands:
    if b[0] not in unique_brands:
        unique_brands.append(b[0])
unique_brands.remove('car')
unique_brands.remove('hyndai kia')
unique_brands.remove('hyundai,')
unique_brands.remove('hyundai.')
unique_brands.remove('kia,')
unique_brands.remove('kia.')
unique_brands.remove('mercedes benz')
unique_brands.remove('mercedes-benz')
unique_brands.remove('nissan.')
unique_brands.remove('problem')
unique_brands.remove('seat')
unique_brands.remove('toyata')
unique_brands.remove('volkwagen')

### Get comments

In [28]:
with open('finalcomments.csv') as f:
    data=[tuple(line) for line in csv.reader(f)]
print(data[-1])

('October  6, 2018 12:55AM', 'henryn', " a few years back, before i returned to work, i played around quite a bit with dvd movies.  as long as it's not copy protected, it is fairly easy to copy to your hard drive, and then change the region coding.  if it's copy protected, that is a different story. and even if you do remove the region coding, you still have the problem of ntsc vs pal.  your dvd player and tv may handle pal -- and then again, maybe not.")


In [29]:
comments = []
for c in data:
    comments.append(c[2])
comments.pop(0)

'comment'

In [30]:
tot_number_of_comments = len(comments)

### Get dictionary of brand name and count

In [36]:
unique_brands_dict = {}
for b in unique_brands:
    for c in comments:
        if b in c:
            if b in unique_brands_dict:
                unique_brands_dict[b] += 1
            else:
                unique_brands_dict[b] = 1

Get top ten brand names

In [37]:
import operator
sorted_brands = sorted(unique_brands_dict.items(), key=operator.itemgetter(1))
sorted_brands.reverse()
sorted_brands[:10]

[('ford', 4115),
 ('dodge', 2709),
 ('chevrolet', 2563),
 ('bmw', 2364),
 ('honda', 2321),
 ('volkswagen', 2083),
 ('audi', 2060),
 ('mercedes', 1734),
 ('toyota', 1598),
 ('cadillac', 1564)]

### Get dictionary of brand name and count of comments with aspirational words

In [38]:
brands_aspiration_dict = {}
for b in unique_brands:
    for c in comments:
        if b in c and\
        ('want' in c\
        or 'choose' in c\
        or 'prefer' in c\
        or 'require' in c\
        or 'needs' in c\
        or 'passion' in c\
        or 'incline' in c\
        or 'tend' in c\
        or 'itch' in c\
        or 'fancy' in c\
        or 'wish' in c\
        or 'great' in c\
        or 'looking' in c\
        or 'like' in c\
        or 'good' in c\
        or 'best' in c\
        or 'nice' in c\
        or 'worth' in c\
        or 'love' in c):
            if b in brands_aspiration_dict:
                brands_aspiration_dict[b] += 1
            else:
                brands_aspiration_dict[b] = 1

### total count of comments with aspirirational words

In [42]:
asp_count = 0
for c in comments:
    if ('want' in c\
    or 'choose' in c\
    or 'prefer' in c\
    or 'require' in c\
    or 'needs' in c\
    or 'passion' in c\
    or 'incline' in c\
    or 'tend' in c\
    or 'itch' in c\
    or 'fancy' in c\
    or 'wish' in c\
    or 'great' in c\
    or 'looking' in c\
    or 'like' in c\
    or 'good' in c\
    or 'best' in c\
    or 'nice' in c\
    or 'worth' in c\
    or 'love' in c):
        asp_count += 1

### Calculate lift for each brand combo

In [44]:
# tot_number_of_comments
# unique_brands_dict
# brands_aspiration_dict
# asp_count

brand_asp_lifts_dict = {}
for b in brands_aspiration_dict:
    
    brand_asp_count = brands_aspiration_dict[b]
    brand_count = unique_brands_dict[b]
    
    lift = (tot_number_of_comments * brand_asp_count) / (brand_count * asp_count)
    brand_asp_lifts_dict[b] = lift

In [45]:
brand_asp_lifts_dict

{'acura': 1.4829089346461937,
 'audi': 1.4591991779167053,
 'bmw': 1.3889266677215755,
 'buick': 1.3203393765488458,
 'cadillac': 1.419295836242414,
 'chevrolet': 1.352475041246998,
 'chrysler': 1.3526003712836996,
 'dodge': 1.349352685951398,
 'ford': 1.3182953184506796,
 'honda': 1.3462213629847801,
 'hyundai': 1.3699064206955167,
 'infiniti': 1.5275401299958844,
 'kia': 1.4837940535409657,
 'lincoln': 1.3941465263552217,
 'mazda': 1.4000464380159718,
 'mercedes': 1.4077002820247473,
 'mercury': 1.4100796295135671,
 'mitsubishi': 1.324676113826253,
 'nissan': 1.4391863616287162,
 'pontiac': 1.3050705294026477,
 'saturn': 1.5569551711115317,
 'sedan': 1.5047919455775787,
 'subaru': 1.3661448035201458,
 'suzuki': 1.14895377219624,
 'toyota': 1.3878398115029988,
 'volkswagen': 1.3494553066145467,
 'volvo': 1.3465528833598301}